In [61]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from ast import literal_eval


In [62]:
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path

nb_dir = Path(os.getcwd())

project_root = nb_dir.parents[1]

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("project_root:", project_root)
print("sys.path[0]:", sys.path[0])

project_root: /data/ephemeral/pro-nlp-generationfornlp-nlp-13
sys.path[0]: /data/ephemeral/pro-nlp-generationfornlp-nlp-13


In [63]:
from src.data.preprocessor import parse_problems_column, add_choices_len
from src.data.data_loader import DataConfig, make_train_valid_dataset
from src.prompt.prompt_registry import PromptRegistry
from src.prompt.prompt_builder import PromptBuilder, PromptConfig
from src.data.data_loader import DataConfig, make_train_valid_dataset
from src.data.tokenizer_wrapper import TokenizerConfig

policy = {
    "system": {4: "fewshot", 5: "fewshot"},
    "user":   {4: "fewshot", 5: "fewshot"},
}

prompt_cfg = PromptConfig(
    policy=policy,
    mode="train",
    verbose=True
)

data_cfg = DataConfig(
    train_path=project_root / "data" / "train.csv",
    valid_ratio=0.1,
    seed=42,
    do_split=True,
)

tokenize_cfg_train = TokenizerConfig(
    max_length=2048,
    truncation=True,
    padding=False,
    add_generation_prompt=False,
)

tokenize_cfg_gen = TokenizerConfig(
    max_length=2048,
    truncation=True,
    padding=False,
    add_generation_prompt=True,
)

MODEL_NAME = "Qwen/Qwen3-14B"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [64]:
ds = make_train_valid_dataset(
    data_cfg=data_cfg,
    prompt_cfg=prompt_cfg,
    tokenize_cfg_train=tokenize_cfg_train,
    tokenize_cfg_gen=tokenize_cfg_gen,
    tokenizer=tokenizer,
)

template loading 완료: system=5, user_4=6, user_5=4
template loading 완료: system=5, user_4=6, user_5=4


Build train messages:   0%|          | 0/1827 [00:00<?, ? examples/s]

Serialize train to text:   0%|          | 0/1827 [00:00<?, ? examples/s]

Build valid messages (teacher forcing):   0%|          | 0/204 [00:00<?, ? examples/s]

Serialize valid to text:   0%|          | 0/204 [00:00<?, ? examples/s]

Build valid_gen messages (prompt only):   0%|          | 0/204 [00:00<?, ? examples/s]

Serialize valid_gen to text (+meta):   0%|          | 0/204 [00:00<?, ? examples/s]

In [65]:
ds['train'][0]

{'id': 'generation-for-nlp-2661',
 'label': 3,
 'text': "<|im_start|>system\n당신은 객관식 문제풀이의 달인입니다.<|im_end|>\n<|im_start|>user\n밑의 예시와 같이 추론과정을 통해 문제를 해결하세요.\n[예시]\n지문:\n성낙송 수원지방법원장이 대법원장 후보로 있다. 성 원장은 법원행정처 사법정책심의관과 공보관, 서울고법 부장판사 등을 지냈다. 양형위원회 초대 상임위원으로 양형 기준 기초를 마련했으며 서울중앙지법에서 일할 때는 성폭력 피해자 증인지원 프로그램을 처음 도입했다.\n질문:\n성낙송 수원지방법원장이 주력해온 분야는 무엇인가?\n선택지:\n1 형사법\n2 민법\n3 상법\n4 행정법\n5 가사법\n추론:\n1. 지문 분석. 성낙송 원장은 '양형 기준 기초' 마련, '성폭력 피해자 증인지원 프로그램' 도입\n2. 개념연결. 지문의 '양형'은 유죄 판결을 받은 피고인에게 형벌의 정도를 정하는 형사 재판의 핵심 절차. 민법, 상법, 행정법 등에서 사용하지 않는 개념\n3. 개념 연결. '성폭력'은 형법상 범죄 행위, 이를 다루는 재판은 형사 재판.\n4. 선택지 대조. 정답은 1번.\n\n### 지문\n“올해 미국 경제에는 태양과 달과 별이 한 줄로 서는 행운이 다가오고 있다.”제이미 다이먼 JP모간체이스 최고경영자(CEO·사진)는 지난주 실적발표 후 투자자들과의 화상 회의에서 이렇게 말했다. 평소 미국 경제를 “조심스럽게 낙관한다”고 말해온 다이먼 CEO는 이날 ‘조심스러운’이라는 단어를 사용하지 않았다. 그는 “실제로 경제 전망이 낙관적이기 때문에 ‘낙관적’이라고 말하는 것”이라며 “대기업, 중소기업, 주식시장, 주택시장 등 어느 한 곳에서도 취약한 부분을 찾기 어렵다”고 덧붙였다.다이먼 CEO뿐 아니다. 월스트리트 대형 은행의 최고 경영진도 잇따라 미국 경제에 대한 장밋빛 전망을 쏟아내고 있다. 기업 대출이 사상 최대 수준으로 늘어났기 때문이다. 미국 중앙은행(Fed)에 

In [66]:
prompt_cfg = PromptConfig(
    policy=policy,
    mode="train",
    verbose=True
)

builder = PromptBuilder(prompt_cfg)

template loading 완료: system=5, user_4=6, user_5=4


In [67]:
data_path = project_root / "data" / "train.csv"
df = pd.read_csv(data_path)
df = df[df['problems'].apply(lambda x: len(literal_eval(x)['choices']) == 5)]

df = parse_problems_column(df)
df = add_choices_len(df)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1239 entries, 792 to 2030
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1239 non-null   object 
 1   paragraph      1239 non-null   object 
 2   question_plus  0 non-null      float64
 3   question       1239 non-null   object 
 4   choices        1239 non-null   object 
 5   answer         1239 non-null   int64  
 6   choices_len    1239 non-null   int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 77.4+ KB


In [68]:
sorted_df = df.sort_values(by='paragraph', key=lambda x: x.str.len(), ascending=False)
sorted_df.iloc[0,:]

id                                         generation-for-nlp-1943
paragraph        서울시가 2030년까지 ‘4대문 안 한양도성’과 ‘강남’ ‘영등포·여의도’ 등을 국...
question_plus                                                  NaN
question         서울시의 2030 도시기본계획에서 강남과 영등포·여의도가 각각 특화된 중심지는 무엇...
choices          [국제업무중심지, 세계적 역사문화중심지, 국제금융중심지, 광역교통망 중심지, 자족기...
answer                                                           1
choices_len                                                      5
Name: 1240, dtype: object

In [69]:
ex = builder.build_message(sorted_df.iloc[0,:])

In [70]:
inputs = tokenizer.apply_chat_template(
	ex['messages'],
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
)

In [71]:
token_count = len(inputs['input_ids'][0]) 

print(f"토큰 수: {token_count}")

토큰 수: 1998


In [72]:
len(tokenizer.decode(inputs['input_ids'][0]))

2824